In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# READ IN SFR, MFR, MIGRATION DATA

### Property File (meta data): 
A property snap shot that has the market by PID which is what we will use to map and extract the markets of interest

### SFR: Read in Haystacks.AI Rental Price Index and Housing Price Index

Haystacks.AI generates their own unique zipcode-based rental price index over time \
These data are available on a monthly time-step from Jan, 2010 to August, 2023 

### Check RPI and HPI for data missingness then subset on target markets
target markets: Atlanta and Cleveland Metropolitan areas (as id'd in the Property File)

### Read in MFR
- Subset on Target Markets
- Check for missingness
- Explore how to aggregate
- Aggregate and combine with SFR

### Read in Migration

In [2]:
meta = pd.read_csv("../data/PropertyFileAugust2023.csv")[['Market', 'Submarket', 'PID', 'propertyname', 'addressall', 'city',
       'state', 'zipcode', 'Longitude', 'Latitude', 'nounits', 'OccupancyDate', 'Occupancy']]

# Subset on markets of interest
meta = meta[meta['Market'].isin(['Cleveland - Akron', 'Atlanta - Urban', 'Atlanta - Suburban'])]
print(meta['Market'].unique())

print('No. unique zip codes in these markets: ' + str(meta['zipcode'].nunique()))
print(meta['zipcode'].dtype) #int
# Change to object so we can subset RPI and HPI (zips are objects in those dataframes)
#meta['zipcode'] = meta['zipcode'].astype('str')
meta.head()

['Atlanta - Suburban' 'Atlanta - Urban' 'Cleveland - Akron']
No. unique zip codes in these markets: 267
int64


,Market,Submarket,PID,propertyname,addressall,city,state,zipcode,Longitude,Latitude,nounits,OccupancyDate,Occupancy
1376,Atlanta - Suburban,1 - Sandy Springs,24_1345530,ARIUM Morgan Falls,8085 Adair Lane,Sandy Springs,GA,30350,-84.361393,33.973211,1180,2023-07-01,0.939831
1377,Atlanta - Suburban,1 - Sandy Springs,24_1393534,North Springs Center,7300 Roswell Road,Atlanta,GA,30328,-84.365632,33.954417,286,NaN,NaN
1378,Atlanta - Suburban,1 - Sandy Springs,24_1465687,North River,8700 Roswell Road,Atlanta,GA,30350,-84.351466,34.001163,322,NaN,NaN
1379,Atlanta - Suburban,1 - Sandy Springs,24_2119,Highland Park,100 Highland Park Trail,Atlanta,GA,30350,-84.342998,33.985181,188,2023-07-01,0.978723
1380,Atlanta - Suburban,1 - Sandy Springs,24_2291,Water's Edge,8601 Roberts Drive,Atlanta,GA,30350,-84.335840,33.994012,208,2023-07-01,0.951923


In [3]:
# Read in SFR
rpi_index = open("../data/SFR/rpi_index.pkl","rb")
rpi = pd.read_pickle(rpi_index)
rpi = pd.DataFrame(rpi)

In [4]:
# Use datetime.to_period() method to extract month and year
rpi['Month_Year'] = rpi['date'].dt.to_period('M')

print(str(rpi['Month_Year'].nunique()) + ' Monthly Periods 2010-Present') # 164 periods, monthly from 2010 to present

print('Dates from: ' + str(min(rpi['Month_Year'])) + ' to ' + str(max(rpi['Month_Year'])))

print(rpi['rental_index'].describe())

print(rpi['census_zcta5_geoid'].dtype)
rpi['census_zcta5_geoid'] = rpi['census_zcta5_geoid'].astype('int64')
#rpi.head()
# Subset on desired market zip codes using all of the zip codes in meta for desired markets
rpi = rpi[rpi['census_zcta5_geoid'].isin(meta['zipcode'].unique())]



164 Monthly Periods 2010-Present
Dates from: 2010-01 to 2023-08
count    1.684308e+06
mean     1.265359e+02
std      3.193228e+01
min      3.426906e+01
25%      1.074084e+02
50%      1.198266e+02
75%      1.388650e+02
max      2.691471e+03
Name: rental_index, dtype: float64
object


In [5]:
print(rpi['census_zcta5_geoid'].nunique())
print(meta['zipcode'].nunique())

239
267


In [6]:
print(rpi['census_zcta5_geoid'].nunique() == meta['zipcode'].nunique())


# Extract unique zc from rpi
unique_zipcodes_rpi = set(rpi['census_zcta5_geoid'].unique())

# Extract unique zc from meta data
unique_zipcodes_meta = set(meta['zipcode'].unique())

# Check if unique values in df1 are in df2
if unique_zipcodes_meta.issubset(unique_zipcodes_rpi):
    print("All zipcode values in the metadata markets for Atlanta and Cleveland are in RPI.")
else:
    print("Not all zipcode values in metadata are in RPI.")
    
res = list(set(meta['zipcode']).difference(rpi['census_zcta5_geoid']))
print(res)

missing_areas = meta[meta['zipcode'].isin(res)]
missing_areas = missing_areas[['Market', 'Submarket', 'zipcode']].drop_duplicates(subset=['zipcode'])
missing_areas.to_csv('../data/SFR/SFR_RPI_missing_market_zipcodes.csv')
rpi.to_csv('../data/SFR/SFR_RPI_targetmarket_subset.csv')

False
Not all zipcode values in metadata are in RPI.
[44056, 44067, 44087, 31833, 44221, 44223, 44224, 44236, 44240, 44241, 44266, 44278, 44303, 44304, 44305, 44306, 44307, 44308, 44310, 44311, 44312, 44313, 44314, 44319, 44320, 44414, 30286, 44685]


### CHECK FOR DATA MISSINGNESS

In [7]:
# Check for months missingness

# Set index to date, then resample by months and compute size of each group
s = rpi.set_index('date').resample('MS').size()
print(s[s == 0].index.tolist()) # no missing month-years

# The size of each date's' corresponds to the number of zipcodes for which there are rent index data in that year-month
s

[]


date
2010-01-01    239
2010-02-01    239
2010-03-01    239
2010-04-01    239
2010-05-01    239
2010-06-01    239
2010-07-01    239
2010-08-01    239
2010-09-01    239
2010-10-01    239
2010-11-01    239
2010-12-01    239
2011-01-01    239
2011-02-01    239
2011-03-01    239
2011-04-01    239
2011-05-01    239
2011-06-01    239
2011-07-01    239
2011-08-01    239
2011-09-01    239
2011-10-01    239
2011-11-01    239
2011-12-01    239
2012-01-01    239
2012-02-01    239
2012-03-01    239
2012-04-01    239
2012-05-01    239
2012-06-01    239
2012-07-01    239
2012-08-01    239
2012-09-01    239
2012-10-01    239
2012-11-01    239
2012-12-01    239
2013-01-01    239
2013-02-01    239
2013-03-01    239
2013-04-01    239
2013-05-01    239
2013-06-01    239
2013-07-01    239
2013-08-01    239
2013-09-01    239
2013-10-01    239
2013-11-01    239
2013-12-01    239
2014-01-01    239
2014-02-01    239
2014-03-01    239
2014-04-01    239
2014-05-01    239
2014-06-01    239
2014-07-01    239
2014-

In [8]:
# Look for missingness
df = rpi.groupby(['census_zcta5_geoid']).count()
print('N unique census_cbsa = ' + str(len(df)))
# Test whether the values in rental index always equal date
result = (df['date'] == df['rental_index']).all()

if result:
    print("All counts (non NaN values) in date equal the counts in rental_index. No missing values")
else:
    print("Not all counts in date equal the counts in Rental Price Index indicating missing values.")
    
rpi.groupby(["date"]).count() # 11241 zip codes with rental indices 

N unique census_cbsa = 239
All counts (non NaN values) in date equal the counts in rental_index. No missing values


,rental_index,census_cbsa_geoid,census_zcta5_geoid,Month_Year
date,,,,
2010-01-01,239,239,239,239
2010-02-01,239,239,239,239
2010-03-01,239,239,239,239
2010-04-01,239,239,239,239
2010-05-01,239,239,239,239
2010-06-01,239,239,239,239
2010-07-01,239,239,239,239
2010-08-01,239,239,239,239
2010-09-01,239,239,239,239


### Read in Haystacks.AI Housing Price Index


In [9]:
hpi_base = open("../data/SFR/hpi_base.pkl","rb")
hpi = pd.read_pickle(hpi_base)
hpi = pd.DataFrame(hpi)

hpi.head(200)
# Use datetime.to_period() method to extract month and year
hpi['Month_Year'] = hpi['period_start'].dt.to_period('M')

print(str(hpi['Month_Year'].nunique()) + ' Monthly Periods 2007-Present') # 164 periods, monthly from 2010 to present

print('Dates from: ' + str(min(hpi['Month_Year'])) + ' to ' + str(max(hpi['Month_Year'])))

print(hpi['price_index'].describe())

200 Monthly Periods 2007-Present
Dates from: 2007-01 to 2023-08
count    2.289507e+06
mean     1.002288e+02
std      2.931869e+01
min      3.486247e+01
25%      8.157063e+01
50%      9.412877e+01
75%      1.133255e+02
max      2.636376e+02
Name: price_index, dtype: float64


In [10]:
print(hpi['census_zcta5_geoid'].dtype)
hpi['census_zcta5_geoid'] = hpi['census_zcta5_geoid'].astype('int64')

# Subset on desired market zip codes using all of the zip codes in meta for desired markets
hpi = hpi[hpi['census_zcta5_geoid'].isin(meta['zipcode'].unique())]

hpi.head()

object


,period_start,period_end,trans_period,coef,price_index,census_zcta5_geoid,census_cbsa_geoid,Month_Year
13956,2007-01-01,2007-01-31,1.0,0.000000,100.000000,30309,12060,2007-01
13957,2007-02-01,2007-02-28,2.0,-0.020081,98.011901,30309,12060,2007-02
13958,2007-03-01,2007-03-31,3.0,-0.022500,97.775099,30309,12060,2007-03
13959,2007-04-01,2007-04-30,4.0,-0.033529,96.702646,30309,12060,2007-04
13960,2007-05-01,2007-05-31,5.0,-0.028352,97.204632,30309,12060,2007-05


In [11]:
print(hpi['census_zcta5_geoid'].nunique() == meta['zipcode'].nunique())

print(hpi['census_zcta5_geoid'].nunique())
print(meta['zipcode'].nunique())
# Extract unique zc from rpi
unique_zipcodes_hpi = set(hpi['census_zcta5_geoid'].unique())

# Check if unique values in df1 are in df2
if unique_zipcodes_meta.issubset(unique_zipcodes_hpi):
    print("All zipcode values in the metadata markets for Atlanta and Cleveland are in HPI.")
else:
    print("Not all zipcode values in metadata are in HPI.")
    
res = list(set(meta['zipcode']).difference(hpi['census_zcta5_geoid']))
print(res)

missing_areas = meta[meta['zipcode'].isin(res)]
missing_areas = missing_areas[['Market', 'Submarket', 'zipcode']].drop_duplicates(subset=['zipcode'])
missing_areas.to_csv('../data/SFR/SFR_HPI_missing_market_zipcodes.csv')
hpi.to_csv('../data/SFR/SFR_HPI_targetmarket_subset.csv')

False
239
267
Not all zipcode values in metadata are in HPI.
[44056, 44067, 44087, 31833, 44221, 44223, 44224, 44236, 44240, 44241, 44266, 44278, 44303, 44304, 44305, 44306, 44307, 44308, 44310, 44311, 44312, 44313, 44314, 44319, 44320, 44414, 30286, 44685]


In [12]:
# # Check if HPI and RPI have same subset of zipcodes
if unique_zipcodes_rpi.issubset(unique_zipcodes_hpi):
    print("All zipcode values in the RPI markets for Atlanta and Cleveland are in HPI.")
else:
    print("Not all zipcode values in RPI are in HPI.")
    
res = list(set(rpi['census_zcta5_geoid']).difference(hpi['census_zcta5_geoid']))
print(res)

All zipcode values in the RPI markets for Atlanta and Cleveland are in HPI.
[]


In [13]:
hpi.head()

,period_start,period_end,trans_period,coef,price_index,census_zcta5_geoid,census_cbsa_geoid,Month_Year
13956,2007-01-01,2007-01-31,1.0,0.000000,100.000000,30309,12060,2007-01
13957,2007-02-01,2007-02-28,2.0,-0.020081,98.011901,30309,12060,2007-02
13958,2007-03-01,2007-03-31,3.0,-0.022500,97.775099,30309,12060,2007-03
13959,2007-04-01,2007-04-30,4.0,-0.033529,96.702646,30309,12060,2007-04
13960,2007-05-01,2007-05-31,5.0,-0.028352,97.204632,30309,12060,2007-05


In [14]:
# Merge HPI and RPI

sfr = pd.merge(hpi, rpi, how = 'left', on=['census_zcta5_geoid', 'Month_Year'])

In [15]:
print(len(hpi))
print(len(rpi))
len(sfr) == len(hpi)

47800
39196


True

In [16]:
# Look for missingness
df = sfr.groupby(['census_zcta5_geoid']).count()
print('N unique zipcodes = ' + str(len(df)))
# Test whether the values in rental index always equal date
result = (df['period_start'] == df['rental_index']).all()

if result:
    print("All counts (non NaN values) in date equal the counts in rental_index. No missing values")
else:
    print("Not all counts in date equal the counts in Rental Price Index indicating missing values.")
    
result = (df['period_start'] == df['price_index']).all()

if result:
    print("All counts (non NaN values) in date equal the counts in housing price_index. No missing values")
else:
    print("Not all counts in date equal the counts in House Price Index indicating missing values.")
    
sfr.groupby(["period_start"]).count() # 11241 zip codes with rental indices 

print('rental price index starts in 2010 while housing price index starts in 2007, \
eliminate dates where there is no RPI as this is our target variable')

sfr.rename(columns={"census_cbsa_geoid_x": "census_cbsa_geoid"}, inplace = True)

sfr = sfr.dropna(subset=['rental_index']).drop(
    columns = ['census_cbsa_geoid_y', 'period_start', 'period_end', 'trans_period'])



N unique zipcodes = 239
Not all counts in date equal the counts in Rental Price Index indicating missing values.
All counts (non NaN values) in date equal the counts in housing price_index. No missing values
rental price index starts in 2010 while housing price index starts in 2007, eliminate dates where there is no RPI as this is our target variable


In [17]:
# Rearrange the dataframe columns
cols = ['Month_Year', 'date', 'census_cbsa_geoid', 'census_zcta5_geoid', 'rental_index', 'price_index', 'coef']
sfr = sfr[cols]

In [18]:
sfr.head()

,Month_Year,date,census_cbsa_geoid,census_zcta5_geoid,rental_index,price_index,coef
36,2010-01,2010-01-01,12060,30309,100.000000,77.979092,-0.261114
37,2010-02,2010-02-01,12060,30309,101.601291,77.507785,-0.261181
38,2010-03,2010-03-01,12060,30309,99.448626,77.293222,-0.275530
39,2010-04,2010-04-01,12060,30309,100.546999,76.653083,-0.283132
40,2010-05,2010-05-01,12060,30309,102.961591,76.577951,-0.257281


# Read in Multi-Family Data

In [19]:
# Read in data
mfr_occ = pd.read_csv("../data/MFR/haystacks_occfile_7-26-2023.csv")
mfr_rent = pd.read_csv("../data/MFR/haystacks_rent_7-17-2023.csv")

In [20]:
# Convert dates to datetime objects
mfr_rent['Period'] = pd.to_datetime(mfr_rent['Period'], format = '%Y-%m-%d') 
mfr_occ['Period'] = pd.to_datetime(mfr_occ['Period'], format = '%m/%d/%Y') 

In [21]:
print(mfr_occ.head())
print(mfr_rent.head())

      PID     Period  Occupancy
0  96_872 2015-01-01       97.9
1  96_872 2015-02-01       97.9
2  96_872 2015-03-01       97.7
3  96_872 2015-04-01       97.7
4  96_872 2015-05-01       97.7
      PID UnitType     Period   Rent
0  1_1004      1BR 2015-01-01  473.0
1  1_1004      1BR 2015-02-01  475.0
2  1_1004      1BR 2015-03-01  474.0
3  1_1004      1BR 2015-04-01  473.0
4  1_1004      1BR 2015-05-01  472.0


In [22]:
print(len(mfr_occ) == len(mfr_rent))
print(len(mfr_occ)) 
print(len(mfr_rent))

mfr = pd.merge(mfr_rent, mfr_occ, how = "left", left_on=['PID', 'Period'], right_on = ['PID', 'Period'])

False
5612309
15181758


In [23]:
# Subset for relevant PID for Atlanta and Cleveland Markets
pid_set = set(meta['PID'].unique())

mfr = mfr[mfr['PID'].isin(pid_set)]

In [24]:
# Merge with meta data to get the zipcodes / market info

mfr = pd.merge(mfr, meta.drop(columns = ['Occupancy', 'OccupancyDate']), left_on="PID", right_on="PID", how = "left")

mfr.head()

,PID,UnitType,Period,Rent,Occupancy,Market,Submarket,propertyname,addressall,city,state,zipcode,Longitude,Latitude,nounits
0,22_1004844,1BR,2015-01-01,1080.774194,93.2,Atlanta - Urban,38 - Sandy Springs/Dunwoody,"Point at Perimeter, The",100 Ashford Gables Drive,Dunwoody,GA,30338,-84.341317,33.935091,366
1,22_1004844,1BR,2015-02-01,1138.064516,93.2,Atlanta - Urban,38 - Sandy Springs/Dunwoody,"Point at Perimeter, The",100 Ashford Gables Drive,Dunwoody,GA,30338,-84.341317,33.935091,366
2,22_1004844,1BR,2015-03-01,1127.903226,93.2,Atlanta - Urban,38 - Sandy Springs/Dunwoody,"Point at Perimeter, The",100 Ashford Gables Drive,Dunwoody,GA,30338,-84.341317,33.935091,366
3,22_1004844,1BR,2015-04-01,1117.741935,92.6,Atlanta - Urban,38 - Sandy Springs/Dunwoody,"Point at Perimeter, The",100 Ashford Gables Drive,Dunwoody,GA,30338,-84.341317,33.935091,366
4,22_1004844,1BR,2015-05-01,1107.580645,91.8,Atlanta - Urban,38 - Sandy Springs/Dunwoody,"Point at Perimeter, The",100 Ashford Gables Drive,Dunwoody,GA,30338,-84.341317,33.935091,366


In [25]:
# Use datetime.to_period() method to extract month and year
mfr['Month_Year'] = mfr['Period'].dt.to_period('M')

print(str(mfr['Month_Year'].nunique()) + ' Monthly Periods 2015-Present') # 164 periods, monthly from 2010 to present

print('Dates from: ' + str(min(mfr['Month_Year'])) + ' to ' + str(max(mfr['Month_Year'])))

rent_desc_stats = mfr.groupby(['Market','Submarket', 'zipcode'])['Rent'].describe()

#print(mfr_rent_merge.groupby(['Market','Submarket'])['Occupancy'].describe().loc[['mean', 'std']])
rent_desc_stats.head()


102 Monthly Periods 2015-Present
Dates from: 2015-01 to 2023-06


count         mean  \
Market             Submarket               zipcode                        
Atlanta - Suburban 1 - Sandy Springs       30328    1224.0  1391.622054   
                                           30350    7406.0  1344.332090   
                   10 - Lithonia - Decatur 30034    1462.0   977.350347   
                                           30035    1836.0   987.323819   
                                           30038    2652.0  1167.213912   

                                                           std         min  \
Market             Submarket               zipcode                           
Atlanta - Suburban 1 - Sandy Springs       30328    292.613269  699.000000   
                                           30350    328.530677  520.000000   
                   10 - Lithonia - Decatur 30034    288.588279  495.000000   
                                           30035    262.329291  513.276596   
                                           30038    315.424761  585.294118   

                                                            25%          50%  \
Market             Submarket               zipcode                             
Atlanta - Suburban 1 - Sandy Springs       30328    1184.942424  1371.742188   
                                           30350    1099.676491  1300.000000   
                   10 - Lithonia - Decatur 30034     775.000000   924.000000   
                                           30035     800.000000   943.690318   
                                           30038     929.571068  1108.266234   

                                                            75%     max  
Market             Submarket               zipcode                       
Atlanta - Suburban 1 - Sandy Springs       30328    1577.250000  2669.0  
                                           30350    1536.804230  2915.0  
                   10 - Lithonia - Decatur 30034    1132.250000  1963.0  
                                           30035    1121.940517  1834.4  
                                           30038    1368.368421  2286.0

In [26]:
print(mfr['zipcode'].nunique() == meta['zipcode'].nunique())

print(mfr['zipcode'].nunique())
print(meta['zipcode'].nunique())
# Extract unique zc from rpi
unique_zipcodes_mfr = set(mfr['zipcode'].unique())

# Check if unique values in df1 are in df2
if unique_zipcodes_meta.issubset(unique_zipcodes_mfr):
    print("All zipcode values in the metadata markets for Atlanta and Cleveland are in MFR.")
else:
    print("Not all zipcode values in metadata are in MFR.")
    
res = list(set(meta['zipcode']).difference(mfr['zipcode']))
print(res)

missing_areas = meta[meta['zipcode'].isin(res)]
missing_areas = missing_areas[['Market', 'Submarket', 'zipcode']].drop_duplicates(subset=['zipcode'])
missing_areas.to_csv('../data/MFR/MFR_RentOcc_missing_market_zipcodes.csv')
mfr.to_csv('../data/MFR/MFR_RentOcc_targetmarket_subset.csv')

False
247
267
Not all zipcode values in metadata are in MFR.
[44057, 31816, 31833, 44131, 44319, 30054, 44414, 30107, 30108, 30127, 30137, 30143, 30184, 30268, 30290, 30517, 30548, 30620, 30666, 44023]


## Aggregate MFR by Month-Year and Zipcode
- calculate median and mean standard deviation for rent price and occupancy
- then calculate percentage change for median and mean

In [27]:
mfr['median_rent'] = mfr.groupby(['zipcode', 'Month_Year'])['Rent'].transform('median')
mfr['mean_rent'] = mfr.groupby(['zipcode', 'Month_Year'])['Rent'].transform('mean')
mfr['std_rent'] = mfr.groupby(['zipcode', 'Month_Year'])['Rent'].transform('std')

mfr['median_occ'] = mfr.groupby(['zipcode', 'Month_Year'])['Occupancy'].transform('median')
mfr['mean_occ'] = mfr.groupby(['zipcode', 'Month_Year'])['Occupancy'].transform('mean')
mfr['std_occ'] = mfr.groupby(['zipcode', 'Month_Year'])['Occupancy'].transform('std')

In [28]:
# drop duplicates and just get the aggregated data
mfr.drop(columns = ['propertyname', 'addressall', 'state', 'Longitude', 
                    'Latitude', 'nounits', 'Rent', 'Occupancy', 'UnitType', 'PID', 'Submarket'], inplace=True)
mfr.drop_duplicates(subset=['Month_Year', 'zipcode'], inplace=True)

In [29]:
mfr.head()

,Period,Market,city,zipcode,Month_Year,median_rent,mean_rent,std_rent,median_occ,mean_occ,std_occ
0,2015-01-01,Atlanta - Urban,Dunwoody,30338,2015-01,1262.000000,1301.634559,249.435422,96.05,95.440,1.522485
1,2015-02-01,Atlanta - Urban,Dunwoody,30338,2015-02,1253.500000,1294.881052,254.135439,96.05,95.488,1.576568
2,2015-03-01,Atlanta - Urban,Dunwoody,30338,2015-03,1268.391892,1301.369265,251.635038,96.05,95.500,1.563356
3,2015-04-01,Atlanta - Urban,Dunwoody,30338,2015-04,1277.250000,1303.582878,241.714960,95.40,94.886,1.602041
4,2015-05-01,Atlanta - Urban,Dunwoody,30338,2015-05,1286.157895,1312.353088,253.113807,94.80,94.424,1.682268


In [30]:
# Calculate percent change

# Make sure dataframe is in order
mfr.sort_values(['zipcode', 'Month_Year'], inplace=True)

mfr['pc_mean_rent'] = mfr.groupby('zipcode')['mean_rent'].pct_change() * 100
mfr['pc_med_rent'] = mfr.groupby('zipcode')['median_rent'].pct_change() * 100
mfr['pc_mean_occ'] = mfr.groupby('zipcode')['mean_occ'].pct_change() * 100
mfr['pc_med_occ'] = mfr.groupby('zipcode')['median_occ'].pct_change() * 100

mfr.reset_index(inplace=True)




In [31]:
mfr.drop(columns = 'index', inplace=True)
mfr.head()

,Period,Market,city,zipcode,Month_Year,median_rent,mean_rent,std_rent,median_occ,mean_occ,std_occ,pc_mean_rent,pc_med_rent,pc_mean_occ,pc_med_occ
0,2015-01-01,Atlanta - Urban,Avondale Estates,30002,2015-01,659.406593,650.976648,88.588183,90.8,91.025,0.45,NaN,NaN,NaN,NaN
1,2015-02-01,Atlanta - Urban,Avondale Estates,30002,2015-02,662.000000,666.380495,92.471641,90.8,91.025,0.45,2.366267,0.393294,0.000000,0.000000
2,2015-03-01,Atlanta - Urban,Avondale Estates,30002,2015-03,653.000000,653.657967,89.066907,91.1,91.250,0.30,-1.909199,-1.359517,0.247185,0.330396
3,2015-04-01,Atlanta - Urban,Avondale Estates,30002,2015-04,644.000000,654.038462,85.153841,90.4,90.725,0.65,0.058210,-1.378254,-0.575342,-0.768386
4,2015-05-01,Atlanta - Urban,Avondale Estates,30002,2015-05,635.000000,645.673077,83.210828,90.4,90.725,0.65,-1.279036,-1.397516,0.000000,0.000000


In [32]:
# Step 1: Group the DataFrame by 'zipcode'
grouped = mfr.groupby('zipcode')

# Step 2: Sort by 'Month_Year' within each group
sorted_df = grouped.apply(lambda group: group.sort_values(by='Month_Year'))

# Step 3: Update the row index 0 of column 'pc_mean_rent' to be 100
sorted_df.loc[sorted_df.groupby(level = 'zipcode').head(1).index, ['pc_mean_rent', 'pc_med_rent', 'pc_mean_occ', 'pc_med_occ']] = 100

# Step 4: Update any other NaN values to be 0
sorted_df.fillna(0, inplace=True)

In [33]:
sorted_df.head()

Period           Market              city  zipcode Month_Year  \
zipcode                                                                       
30002   0 2015-01-01  Atlanta - Urban  Avondale Estates    30002    2015-01   
        1 2015-02-01  Atlanta - Urban  Avondale Estates    30002    2015-02   
        2 2015-03-01  Atlanta - Urban  Avondale Estates    30002    2015-03   
        3 2015-04-01  Atlanta - Urban  Avondale Estates    30002    2015-04   
        4 2015-05-01  Atlanta - Urban  Avondale Estates    30002    2015-05   

           median_rent   mean_rent   std_rent  median_occ  mean_occ  std_occ  \
zipcode                                                                        
30002   0   659.406593  650.976648  88.588183        90.8    91.025     0.45   
        1   662.000000  666.380495  92.471641        90.8    91.025     0.45   
        2   653.000000  653.657967  89.066907        91.1    91.250     0.30   
        3   644.000000  654.038462  85.153841        90.4    90.725     0.65   
        4   635.000000  645.673077  83.210828        90.4    90.725     0.65   

           pc_mean_rent  pc_med_rent  pc_mean_occ  pc_med_occ  
zipcode                                                        
30002   0    100.000000   100.000000   100.000000  100.000000  
        1      2.366267     0.393294     0.000000    0.000000  
        2     -1.909199    -1.359517     0.247185    0.330396  
        3      0.058210    -1.378254    -0.575342   -0.768386  
        4     -1.279036    -1.397516     0.000000    0.000000

In [34]:
# Create a new column 'cumulative_sum' that represents the cumulative sum within each group
sorted_df[['mfr_mean_rent_index', 'mfr_med_rent_index', 'mfr_mean_occ_index', 'mfr_med_occ_index']] = \
sorted_df[['pc_mean_rent','pc_med_rent', 'pc_mean_occ', 'pc_med_occ']].groupby(level='zipcode').cumsum()

# Step 3: Update the row index 0 of column 'pc_mean_rent' to be 100
sorted_df.loc[sorted_df.groupby(level = 'zipcode').head(1).index, \
              ['mfr_mean_rent_index', 'mfr_med_rent_index', 'mfr_mean_occ_index', 'mfr_med_occ_index']] = 100


In [35]:
sorted_df.head(200)

Period              Market              city  zipcode  \
zipcode                                                                 
30002   0   2015-01-01     Atlanta - Urban  Avondale Estates    30002   
        1   2015-02-01     Atlanta - Urban  Avondale Estates    30002   
        2   2015-03-01     Atlanta - Urban  Avondale Estates    30002   
        3   2015-04-01     Atlanta - Urban  Avondale Estates    30002   
        4   2015-05-01     Atlanta - Urban  Avondale Estates    30002   
        5   2015-06-01     Atlanta - Urban  Avondale Estates    30002   
        6   2015-07-01     Atlanta - Urban  Avondale Estates    30002   
        7   2015-08-01     Atlanta - Urban  Avondale Estates    30002   
        8   2015-09-01     Atlanta - Urban  Avondale Estates    30002   
        9   2015-10-01     Atlanta - Urban  Avondale Estates    30002   
        10  2015-11-01     Atlanta - Urban  Avondale Estates    30002   
        11  2015-12-01     Atlanta - Urban  Avondale Estates    30002   
        12  2016-01-01     Atlanta - Urban  Avondale Estates    30002   
        13  2016-02-01     Atlanta - Urban  Avondale Estates    30002   
        14  2016-03-01     Atlanta - Urban  Avondale Estates    30002   
        15  2016-04-01     Atlanta - Urban  Avondale Estates    30002   
        16  2016-05-01     Atlanta - Urban  Avondale Estates    30002   
        17  2016-06-01     Atlanta - Urban  Avondale Estates    30002   
        18  2016-07-01     Atlanta - Urban  Avondale Estates    30002   
        19  2016-08-01     Atlanta - Urban  Avondale Estates    30002   
        20  2016-09-01     Atlanta - Urban  Avondale Estates    30002   
        21  2016-10-01     Atlanta - Urban  Avondale Estates    30002   
        22  2016-11-01     Atlanta - Urban  Avondale Estates    30002   
        23  2016-12-01     Atlanta - Urban  Avondale Estates    30002   
        24  2017-01-01     Atlanta - Urban  Avondale Estates    30002   
        25  2017-02-01     Atlanta - Urban  Avondale Estates    30002   
        26  2017-03-01     Atlanta - Urban  Avondale Estates    30002   
        27  2017-04-01     Atlanta - Urban  Avondale Estates    30002   
        28  2017-05-01     Atlanta - Urban  Avondale Estates    30002   
        29  2017-06-01     Atlanta - Urban  Avondale Estates    30002   
        30  2017-07-01     Atlanta - Urban  Avondale Estates    30002   
        31  2017-08-01     Atlanta - Urban  Avondale Estates    30002   
        32  2017-09-01     Atlanta - Urban  Avondale Estates    30002   
        33  2017-10-01     Atlanta - Urban  Avondale Estates    30002   
        34  2017-11-01     Atlanta - Urban  Avondale Estates    30002   
        35  2017-12-01     Atlanta - Urban  Avondale Estates    30002   
        36  2018-01-01     Atlanta - Urban  Avondale Estates    30002   
        37  2018-02-01     Atlanta - Urban  Avondale Estates    30002   
        38  2018-03-01     Atlanta - Urban  Avondale Estates    30002   
        39  2018-04-01     Atlanta - Urban  Avondale Estates    30002   
        40  2018-05-01     Atlanta - Urban  Avondale Estates    30002   
        41  2018-06-01     Atlanta - Urban  Avondale Estates    30002   
        42  2018-07-01     Atlanta - Urban  Avondale Estates    30002   
        43  2018-08-01     Atlanta - Urban  Avondale Estates    30002   
        44  2018-09-01     Atlanta - Urban  Avondale Estates    30002   
        45  2018-10-01     Atlanta - Urban  Avondale Estates    30002   
        46  2018-11-01     Atlanta - Urban  Avondale Estates    30002   
        47  2018-12-01     Atlanta - Urban  Avondale Estates    30002   
        48  2019-01-01     Atlanta - Urban  Avondale Estates    30002   
        49  2019-02-01     Atlanta - Urban  Avondale Estates    30002   
        50  2019-03-01     Atlanta - Urban  Avondale Estates    30002   
        51  2019-04-01     Atlanta - Urban  Avondale Estates    30002   
        52  2019-05-01     Atlanta - Urban  Avondale Estates    30002   
 

In [36]:
# Reset 'zipcode' as a regular column
sorted_df = sorted_df.droplevel('zipcode')


In [37]:
sorted_df.head()

,Period,Market,city,zipcode,Month_Year,median_rent,mean_rent,std_rent,median_occ,mean_occ,std_occ,pc_mean_rent,pc_med_rent,pc_mean_occ,pc_med_occ,mfr_mean_rent_index,mfr_med_rent_index,mfr_mean_occ_index,mfr_med_occ_index
0,2015-01-01,Atlanta - Urban,Avondale Estates,30002,2015-01,659.406593,650.976648,88.588183,90.8,91.025,0.45,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,2015-02-01,Atlanta - Urban,Avondale Estates,30002,2015-02,662.000000,666.380495,92.471641,90.8,91.025,0.45,2.366267,0.393294,0.000000,0.000000,102.366267,100.393294,100.000000,100.000000
2,2015-03-01,Atlanta - Urban,Avondale Estates,30002,2015-03,653.000000,653.657967,89.066907,91.1,91.250,0.30,-1.909199,-1.359517,0.247185,0.330396,100.457068,99.033777,100.247185,100.330396
3,2015-04-01,Atlanta - Urban,Avondale Estates,30002,2015-04,644.000000,654.038462,85.153841,90.4,90.725,0.65,0.058210,-1.378254,-0.575342,-0.768386,100.515278,97.655523,99.671842,99.562010
4,2015-05-01,Atlanta - Urban,Avondale Estates,30002,2015-05,635.000000,645.673077,83.210828,90.4,90.725,0.65,-1.279036,-1.397516,0.000000,0.000000,99.236243,96.258008,99.671842,99.562010


In [38]:
final = pd.merge(sfr, sorted_df, how = 'left', left_on = ['Month_Year', 'census_zcta5_geoid'], right_on=['Month_Year', 'zipcode'])

In [39]:
final.head(100)

,Month_Year,date,census_cbsa_geoid,census_zcta5_geoid,rental_index,price_index,coef,Period,Market,city,zipcode,median_rent,mean_rent,std_rent,median_occ,mean_occ,std_occ,pc_mean_rent,pc_med_rent,pc_mean_occ,pc_med_occ,mfr_mean_rent_index,mfr_med_rent_index,mfr_mean_occ_index,mfr_med_occ_index
0,2010-01,2010-01-01,12060,30309,100.000000,77.979092,-0.261114,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-02,2010-02-01,12060,30309,101.601291,77.507785,-0.261181,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-03,2010-03-01,12060,30309,99.448626,77.293222,-0.275530,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-04,2010-04-01,12060,30309,100.546999,76.653083,-0.283132,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-05,2010-05-01,12060,30309,102.961591,76.577951,-0.257281,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2010-06,2010-06-01,12060,30309,101.057081,76.409740,-0.276402,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2010-07,2010-07-01,12060,30309,101.196260,75.980188,-0.295150,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2010-08,2010-08-01,12060,30309,101.151763,75.565009,-0.294061,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2010-09,2010-09-01,12060,30309,101.636364,75.049468,-0.317128,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2010-10,2010-10-01,12060,30309,101.793769,74.565248,-0.322457,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
final.rename(columns={'rental_index':'sfr_rental_index', 
                     'price_index': 'sfr_price_index',
                     'median_rent': 'mfr_med_rent', 
                     'mean_rent':'mfr_mean_rent', 
                     'std_rent':'mfr_std_rent',
                     'median_occ':'mfr_med_occ', 
                     'mean_occ':'mfr_mean_occ', 
                     'std_occ':'mfr_std_occ',
                     'mean_rent_index': 'mfr_mean_rent_index',
                      'med_rent_index': 'mfr_med_rent_index',
                      'mean_occ_index': 'mfr_mean_occ_index',
                      'med_occ_index': 'mfr_med_occ_index'}, inplace=True)

In [41]:
final.drop(columns = ['Period', 'date', 'zipcode', 'pc_mean_rent', 'pc_med_rent', 'pc_mean_occ', 'pc_med_occ'], inplace=True)

In [42]:
final.head(200)

,Month_Year,census_cbsa_geoid,census_zcta5_geoid,sfr_rental_index,sfr_price_index,coef,Market,city,mfr_med_rent,mfr_mean_rent,mfr_std_rent,mfr_med_occ,mfr_mean_occ,mfr_std_occ,mfr_mean_rent_index,mfr_med_rent_index,mfr_mean_occ_index,mfr_med_occ_index
0,2010-01,12060,30309,100.000000,77.979092,-0.261114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-02,12060,30309,101.601291,77.507785,-0.261181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-03,12060,30309,99.448626,77.293222,-0.275530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-04,12060,30309,100.546999,76.653083,-0.283132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-05,12060,30309,102.961591,76.577951,-0.257281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2010-06,12060,30309,101.057081,76.409740,-0.276402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2010-07,12060,30309,101.196260,75.980188,-0.295150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2010-08,12060,30309,101.151763,75.565009,-0.294061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2010-09,12060,30309,101.636364,75.049468,-0.317128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2010-10,12060,30309,101.793769,74.565248,-0.322457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
final.to_csv('../data/SFRMFR_combined.csv')

# Read in Migration Data

In [59]:
mig_ata = pd.read_csv('../data/Migration/area_migration_ga_zip.csv')
mig_clv = pd.read_csv('../data/Migration/haystacks_cleveland_market_tract_migration.csv')

In [56]:
mig_ata = mig_ata[mig_ata['us_zip'].isin(meta['zipcode'])]
mig_clv = mig_clv[mig_clv['us_county_id'].isin(meta['zipcode'])]

In [57]:
if set(meta['zipcode']).issubset(set(mig_ata['us_zip'])):

    print("All zipcode values in the migration markets for Atlanta meta.")
else:
    print("Not all zipcode values in META are in HPI.")
    
res = list(set(mig_ata['us_zip']).difference(meta['zipcode']))
print(res)

Not all zipcode values in RPI are in HPI.
[]


In [60]:
mig_clv.head()

,location_id,area,us_state_id,us_state,us_cbsa_id,us_cbsa,us_county_id,us_county,us_tract_id,us_tract,observation_start_date,observation_end_date,student_population_fraction,population,netflow_estimated,inflow_estimated,outflow_estimated,netflow_estimated_normalized,inflow_estimated_normalized,outflow_estimated_normalized,cumulative_netflow_estimated,median_income_inflow,median_income,median_income_difference,median_age_inflow,median_age,median_age_difference,confidence_score
0,39093097500,6683941,39,Ohio,17460,"Cleveland-Elyria, OH Metro Area",39093,"Lorain County, OH",39093097500,39093097500,2023-06-01,2023-06-30,0.0259,6400,-10,21,31,-9,20,29,-942,28943.1602,34511,-5567.8398,23.5333,45.5,-21.9667,1
1,39035115900,1282306,39,Ohio,17460,"Cleveland-Elyria, OH Metro Area",39035,"Cuyahoga County, OH",39035115900,39035115900,2023-06-01,2023-06-30,0.0407,2798,-1,7,8,-1,7,7,-212,24193.9863,41414,-17220.0137,26.4410,48.0,-21.5590,1
2,39093070200,8042626,39,Ohio,17460,"Cleveland-Elyria, OH Metro Area",39093,"Lorain County, OH",39093070200,39093070200,2023-06-01,2023-06-30,0.0247,1900,0,9,9,0,8,8,-47,32141.5000,51141,-18999.5000,26.7538,42.6,-15.8462,1
3,39093013101,16799130,39,Ohio,17460,"Cleveland-Elyria, OH Metro Area",39093,"Lorain County, OH",39093013101,39093013101,2023-06-01,2023-06-30,0.0279,4811,2,21,19,2,20,18,252,49936.2930,103929,-53992.7070,28.7687,44.5,-15.7312,1
4,39035119900,571774,39,Ohio,17460,"Cleveland-Elyria, OH Metro Area",39035,"Cuyahoga County, OH",39035119900,39035119900,2023-06-01,2023-06-30,0.0326,1440,1,10,9,1,9,8,-83,45377.1016,19351,26026.0996,27.7050,52.0,-24.2950,1


In [55]:
mig_ata.head()

,location_id,area,us_state_id,us_state,us_zip_id,us_zip,observation_start_date,observation_end_date,student_population_fraction,population,netflow_estimated,inflow_estimated,outflow_estimated,netflow_estimated_normalized,inflow_estimated_normalized,outflow_estimated_normalized,cumulative_netflow_estimated,median_income_inflow,median_income,median_income_difference,median_age_inflow,median_age,median_age_difference,confidence_score
1,30011,69386929,13,Georgia,30011,30011,2023-06-01,2023-06-30,0.0395,16943,52,157,105,49,147,98,1673,69231.7891,73195,-3963.2146,31.4163,33.6,-2.1837,1
10,30121,152846499,13,Georgia,30121,30121,2023-06-01,2023-06-30,0.0402,25285,13,136,123,12,127,115,-929,59640.3438,63839,-4198.6553,32.0620,37.6,-5.5380,1
15,30327,44199823,13,Georgia,30327,30327,2023-06-01,2023-06-30,0.0301,23166,-15,158,173,-14,147,161,489,67632.3594,162412,-94779.6406,31.0382,46.6,-15.5618,1
16,30062,67611749,13,Georgia,30062,30062,2023-06-01,2023-06-30,0.0415,63695,-10,326,336,-9,304,314,944,73176.3594,115953,-42776.6406,32.3359,41.5,-9.1641,1
20,30213,168347724,13,Georgia,30213,30213,2023-06-01,2023-06-30,0.0389,40126,38,263,225,35,245,210,2381,49280.8750,72817,-23536.1250,29.5876,33.4,-3.8124,1
